In [9]:
import pymongo
import os,csv,sys
import openslide

client = pymongo.MongoClient('localhost',27017)
slide_root = '/GLOBAL_SCRATCH/ADRC/'  ##Base Path for Slides
dsa_slide_db = client['ADRC_DSA_Slide_DB']  ### These need to be configured for the specific project
dsa_load_errors_db = client['DSA_LoadErrors']

In [2]:
def find_rawslide_lists( slide_root_path ):
        """project_name is passed along with the potentially more than one root image path for ndpi files"""
        slide_files = []

        slide_root_path  = slide_root_path.rstrip('/')
        print slide_root_path
        for dpath, dnames, fnames in os.walk( slide_root_path, followlinks=True):
                
                for file in fnames:
                    if '.ndpi' in file or '.svs' in file:
                                slide_files.append(dpath +'/'+file)
        print len(slide_files),"SVS or NDPI files were located"
        return slide_files

In [4]:
import dsa_mongo_common_functions as dsa
import cdsa_loader_helper_functions as cdsa_helpers
reload(dsa)
reload(cdsa_helpers)
### To generalize this, need to describe organization, most common will be  PATIENT/STAIN_TYPE as subdirectories
subj_dir_list = [x for x in os.listdir(slide_root) if os.path.isdir(os.path.join(slide_root,x))]
print len(subj_dir_list),"Potential Patient directories were identified"

96 Potential Patient directories were identified


In [ ]:
collection_ID = 'ADRC'
for sd in subj_dir_list:
    print sd,"is being processed",
    curr_svs_slide_list = find_rawslide_lists(  os.path.join(slide_root,sd)  )
    slides_processed = newly_processed = dup_slide = rescanned_slides =  0
    for sld in curr_svs_slide_list:

        slide_name = os.path.basename(sld)
        qry = dsa_slide_db[collection_ID].find_one( {'slide_name':slide_name})
        #print qry
        if not qry:
            fs = os.path.getsize(sld)
            #print fs
            #md5Checksum = dsa.md5sum(sld)
            (openslide_could_open, width, height, filesize, orig_resolution, slide_name,md5, sld_properties) = cdsa_helpers.openslide_test_file_mongo( sld, 'ndpi', client)
            if openslide_could_open:
                prep_type = 'Unknown'
                slide_metadata = { 'slide_w_path': sld, 'slide_name': slide_name, 'file_size':fs, 'width':width, 'height':height,
                                 'orig_resolution': orig_resolution, 'sld_properties': cdsa_helpers.clean_openslide_keys ( sld_properties), 'slide_md5': md5, 'prep_type': prep_type
                                 }
                dsa_slide_db[collection_ID].insert_one(slide_metadata)
                newly_processed +=1 
            else:
                print "UNABLE TO OPEN FILE??",sld
                ###Need to flag/load this in to an error database

        else:
            #print "This slide was loaded?",slide_name
            fs = os.path.getsize(sld)
            ## Double check if file size matches
#             if qry['file_size'] != fs:
#                 #print "File size mismatch??",fs,qry['file_size'],qry['slide_w_path'],sld
#                 load_errors_db['rescanned_slides'].insert_one( {'loaded_slide': qry['slide_w_path'], 'rescanned_slide': sld}             )
#                 rescanned_slides +=1 
#             else:
#                 dup_slide +=1

    
        slides_processed +=1        
        output = "Total Processed: %d  Newly Processed: %d Dup Slides or Already Loaded: %d  RESCANNED Slides %d" % (slides_processed, newly_processed, dup_slide, rescanned_slides )
        dsa.LinePrinter(output)


ADRC61-124 is being processed /GLOBAL_SCRATCH/ADRC/ADRC61-124
106 SVS or NDPI files were located
Total Processed: 106  Newly Processed: 0 Dup Slides or Already Loaded: 0  RESCANNED Slides 0ADRC50-10 is being processed /GLOBAL_SCRATCH/ADRC/ADRC50-10
58 SVS or NDPI files were located
Total Processed: 58  Newly Processed: 0 Dup Slides or Already Loaded: 0  RESCANNED Slides 0ADRC61-128 is being processed /GLOBAL_SCRATCH/ADRC/ADRC61-128
99 SVS or NDPI files were located
Total Processed: 99  Newly Processed: 0 Dup Slides or Already Loaded: 0  RESCANNED Slides 0ADRC50-40 is being processed /GLOBAL_SCRATCH/ADRC/ADRC50-40
59 SVS or NDPI files were located
Total Processed: 59  Newly Processed: 0 Dup Slides or Already Loaded: 0  RESCANNED Slides 0ADRC51-60 is being processed /GLOBAL_SCRATCH/ADRC/ADRC51-60
10 SVS or NDPI files were located
Total Processed: 10  Newly Processed: 0 Dup Slides or Already Loaded: 0  RESCANNED Slides 0@eaDir is being processed /GLOBAL_SCRATCH/ADRC/@eaDir
0 SVS or NDPI f

Verify failed with: (u"Can't validate JPEG for directory 0: Expected marker at 4294972345, found none",)
Verify failed with:

Openslide returned an error /GLOBAL_SCRATCH/ADRC/ADRC51-06/BIELS/ADRC51-06_10_BIELS.ndpi
Openslide returned an error /GLOBAL_SCRATCH/ADRC/ADRC51-06/BIELS/ADRC51-06_10_BIELS.ndpi
SHIT IT DIED!
UNABLE TO OPEN FILE?? /GLOBAL_SCRATCH/ADRC/ADRC51-06/BIELS/ADRC51-06_10_BIELS.ndpi
Total Processed: 27  Newly Processed: 26 Dup Slides or Already Loaded: 0  RESCANNED Slides 0

 (u"Can't validate JPEG for directory 0: Expected marker at 4294972181, found none",)
Verify failed with:

Openslide returned an error /GLOBAL_SCRATCH/ADRC/ADRC51-06/BIELS/ADRC51-06_8_BIELS.ndpi
Openslide returned an error /GLOBAL_SCRATCH/ADRC/ADRC51-06/BIELS/ADRC51-06_8_BIELS.ndpi
SHIT IT DIED!
UNABLE TO OPEN FILE?? /GLOBAL_SCRATCH/ADRC/ADRC51-06/BIELS/ADRC51-06_8_BIELS.ndpi
Total Processed: 31  Newly Processed: 29 Dup Slides or Already Loaded: 0  RESCANNED Slides 0

In [ ]:



sys.exit()
for sd in subj_dir_list:
    print sd,"is being processed"
    curr_svs_slide_list = find_rawslide_lists(  os.path.join(slide_root,sd)  )
    slides_processed = newly_processed = dup_slide = rescanned_slides =  0
    for sld in curr_svs_slide_list:
        #print sld
        ## Building the basic dictionary to look up.... for now I'll just index on the full slide name... may change this later though
        slide_name = os.path.basename(sld)
        ### check the DB to see if the slide is indexed already
        qry = dsa_slide_db[collection_ID].find_one( {'slide_name':slide_name})
        if not qry:
            fs = os.path.getsize(sld)
            #md5Checksum = dsa.md5sum(sld)
            (openslide_could_open, width, height, filesize, orig_resolution, slide_name,md5, sld_properties) = cdsa_helpers.openslide_test_file_mongo( sld, 'svs', client)
            if openslide_could_open:
                #print width,height,filesize,orig_resolution,md5, sld_properties
                ### Need to determine the stain type.... if that's even possible
                prep_type = 'Unknown'
                slide_metadata = { 'slide_w_path': sld, 'slide_name': slide_name, 'file_size':fs, 'width':width, 'height':height,
                                 'orig_resolution': orig_resolution, 'sld_properties': cdsa_helpers.clean_openslide_keys ( sld_properties), 'slide_md5': md5, 'prep_type': prep_type
                                 }
                #print slide_metadata
                dsa_slide_db[collection_ID].insert_one(slide_metadata)
                newly_processed +=1 
            else:
                print "UNABLE TO OPEN FILE??",sld
                sys.exit()
        else:
            ### See if the other slide matches
            #print "Image seems to have been loaded?"
            

        slides_processed +=1        
        output = "Total Processed: %d  Newly Processed: %d Dup Slides or Already Loaded: %d  RESCANNED Slides %d" % (slides_processed, newly_processed, dup_slide, rescanned_slides )
        dsa.LinePrinter(output)


In [ ]:
wtf = '/TCGA_MIRROR/tcga-data.nci.nih.gov/tcgafiles/ftp_auth/distro_ftpusers/anonymous/tumor/coad/bcr/nationwidechildrens.org/diagnostic_images/slide_images/nationwidechildrens.org_COAD.diagnostic_images.Level_1.138.9.0/TCGA-AA-3489-01Z-00-DX1.AE299B70-B14C-4FFE-B1F9-38B2EB267FA9.svs'

In [ ]:
import openslide
import cdsa_loader_helper_functions as cdsa_helpers
reload(cdsa_helpers)
cdsa_helpers.openslide_test_file_mongo(wtf,'svs',client)
print "HI"
sys.exit()
        
im = openslide.open_slide(wtf)
(width, height) = im.dimensions
base_file_name = os.path.basename(wtf)
filesize = os.path.getsize(wtf)
if True:
    try:
        orig_resolution = im.properties['aperio.AppMag']
    except:
        orig_resolution = 'UnknownSVSError'
#md5 = md5Checksum(full_file_path)
md5 = None
slide_name = os.path.basename(wtf)
sld_properties = im.properties
print im.properties

In [ ]:
print os.getcwd()